In [78]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [79]:
import os
import numpy as np
import pandas as pd 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
import matplotlib.pyplot as plt

import torch, torch.nn.functional as F
from torch import ByteTensor, DoubleTensor, FloatTensor, HalfTensor, LongTensor, ShortTensor, Tensor
from torch import nn, optim, as_tensor
from torch.utils.data import BatchSampler, DataLoader, Dataset, Sampler, TensorDataset
from torch.nn.utils import weight_norm

from collections.abc import Iterable
from tqdm import tqdm_notebook as tqdm

In [284]:
import gc

## Data

In [80]:
ref_train_x = pd.read_csv('data/train_features.csv')
ref_train_y = pd.read_csv('data/train_targets_scored.csv')
ref_train_y2 = pd.read_csv('data/train_targets_nonscored.csv')

ref_test_x = pd.read_csv('data/test_features.csv')
smplsub = pd.read_csv('data/sample_submission.csv')

In [81]:
# all IDs unique?
((ref_train_x['sig_id'].value_counts()) == 1).all()

True

In [82]:
train_y_all = pd.merge(ref_train_y, ref_train_y2, on='sig_id')
train_y_all['ctrl'] = (ref_train_x['cp_type'] == 'ctl_vehicle').astype(int)
train_y_all['other'] = (train_y_all.loc[:,ref_train_y2.columns].sum(axis=1) > 0).astype(int)
train_y_all['zero'] = (train_y_all.sum(axis=1) == 0).astype(int)

NameError: name 'ref_train_y2' is not defined

In [ ]:
mod_train_y = train_y_all.loc[:,list(ref_train_y.columns)+['ctrl', 'zero', 'other']]

In [ ]:
def onehot_col(df, col):
    enc = pd.get_dummies(df[col])
    enc.columns = [f"{col}_{n}" for n in enc.columns]
    df = df.drop(col, axis=1)
    df = df.join(enc)
    return df

def prep_data(df, cols, func=onehot_col):
    for i in cols: df = func(df, i)
    return df

In [ ]:
# # non_ctrl only:
# _ref_train_x_trt = ref_train_x[ref_train_x['cp_type'] != 'ctl_vehicle'].drop('cp_type', axis=1)
# _ref_test_x_trt = ref_test_x[ref_test_x['cp_type'] != 'ctl_vehicle'].drop('cp_type', axis=1)

# non_zero only:
_ref_train_x_trt = ref_train_x[ref_train_x.sum(axis=1) > 0].drop('cp_type', axis=1)
_ref_test_x_trt = ref_test_x[ref_test_x.sum(axis=1) > 0].drop('cp_type', axis=1)

# normal prep
trt_ref_train_x = prep_data(_ref_train_x_trt, cols=['cp_time', 'cp_dose'])
trt_ref_test_x = prep_data(_ref_test_x_trt, cols=['cp_time', 'cp_dose'])

x_fts = trt_ref_train_x.columns[1:]
y_fts = ref_train_y.columns[1:]

trt_trnval_df_rdy = pd.merge(trt_ref_train_x, ref_train_y, on='sig_id')
trt_test_df_rdy = trt_ref_test_x

In [ ]:
print(_ref_train_x_trt.shape, ref_train_x.shape)

In [ ]:
trt_test_df_rdy.head()

In [ ]:
def get_label_stratified_val_idxs(df, val_size=0.1, rnd=0):
    
    arr = df.to_numpy()

    X = arr[:,0]
    y = arr[:,1:] # this works irrespective of whether labels are space- or comma-separated
    
    ### sklearn.model_selection.StratifiedKFold
    sss = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=rnd)
    
    for train_index, val_index in sss.split(X, y):
        trn_idxs = train_index
        val_idxs = val_index

    data_report(df, trn_idxs, val_idxs)
    return trn_idxs, val_idxs

def finalize_df(df, targets, as_multi=True): 
    # Select and fuse labels into target column (space separated)
    df_slct = df[[df.columns[0]] + targets]
    if as_multi:
        df_out = np.array([[df_slct.values[i][0], ' '.join(str(x) for x in df_slct.values[i][1:])] for i in range(len(df_slct))])
        return pd.DataFrame(df_out, columns = ["ID", "Target"])
    else: 
        df_out = np.array(df_slct)
        if len(targets) == 1: return pd.DataFrame(df_out, columns = ["ID", 'Target'])
        else: return pd.DataFrame(df_out, columns = ["ID"] + targets)

def data_report(df, trn_idxs, val_idxs, test_csv=None):
    trnval = df
    if len(trnval.columns) != 2:
        print(f"Multilabel csv with comma-separated labels detected!\n")
        trnval = finalize_df(trnval, targets=list(trnval.columns)[1:])
    print(f"""Train label-distribution:\n"""
          f"""{trnval['Target'][trn_idxs].value_counts()}\n"""
          f"""Total: {len(trn_idxs)}\n""")
    print(f"""Val label-distribution:\n"""
          f"""{trnval['Target'][val_idxs].value_counts()}\n"""
          f"""Total: {len(val_idxs)}""")

In [ ]:
trn_idxs, val_idxs = get_label_stratified_val_idxs(_ref_train_x_trt.iloc[:,:3], val_size=0.1, rnd=0)

In [ ]:
# TODO:
# - normlize fts
# - embed cat fts

## Dataloader 

In [83]:
class MOA_data:
    def __init__(self, x_fts, y_fts, bs=512):
        self.x_fts, self.y_fts, self.bs = x_fts, y_fts, bs
    
    def embed(df):
        return df

    def create(self, df, val_idxs, test=None): 
        train = df.drop(val_idxs)
        valid = df.loc[val_idxs]
        for ID in valid['sig_id']: assert ID not in list(train['sig_id']) 
        self.train_ds = MOA_ds(train, self.x_fts, self.y_fts)
        self.valid_ds = MOA_ds(valid, self.x_fts, self.y_fts)
        
        self.train_dl = DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)
        self.valid_dl = DataLoader(self.valid_ds, batch_size=self.bs, shuffle=False)
        self.fix_dl = DataLoader(self.train_ds, batch_size=self.bs, shuffle=False)
        
        if test is not None:
            self.test_ds = MOA_ds(test, self.x_fts, y_fts, test=True)
            self.test_dl = DataLoader(self.test_ds, batch_size=self.bs, shuffle=False)

class MOA_ds(Dataset):
    def __init__(self, df, x_fts, y_fts, test=False):
        if test: self.x, self.y = df[x_fts].to_numpy(), np.zeros((df.shape[0], len(y_fts)))
        else: self.x, self.y = df[x_fts].to_numpy(), df[y_fts].to_numpy()
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return [torch.tensor(self.x[idx, :], dtype=torch.float),
                torch.tensor(self.y[idx, :], dtype=torch.float)]

In [84]:
data = MOA_data(x_fts, y_fts)
# data.create(trnval_df_rdy, val_idxs, test=test_df_rdy)
data.create(trt_trnval_df_rdy, val_idxs, test=trt_test_df_rdy)

## Model 

In [85]:
def ifnone(a,b):
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

def listify(p=None, q=None):
    "Make `p` listy and the same length as `q`."
    if p is None: p=[]
    elif isinstance(p, str):          p = [p]
    elif not isinstance(p, Iterable): p = [p]
    #Rank 0 tensors in PyTorch are Iterable but don't have a length.
    else:
        try: a = len(p)
        except: p = [p]
    n = q if type(q)==int else len(p) if q is None else len(q)
    if len(p)==1: p = p * n
    assert len(p)==n, f'List len mismatch ({len(p)} vs {n})'
    return list(p)

def emb_sz_rule(n_cat:int)->int: return min(600, round(1.6 * n_cat**0.56))

def def_emb_sz(classes, n, sz_dict=None):
    "Pick an embedding size for `n` depending on `classes` if not given in `sz_dict`."
    sz_dict = ifnone(sz_dict, {})
    n_cat = len(classes[n])
    sz = sz_dict.get(n, int(emb_sz_rule(n_cat)))  # rule of thumb
    return n_cat,sz

def get_emb_szs(self, sz_dict=None):
    "Return the default embedding sizes suitable for this data or takes the ones in `sz_dict`."
    return [def_emb_sz(self.classes, n, sz_dict) for n in self.cat_names]

def embedding(ni,nf):
    "Create an embedding layer."
    emb = nn.Embedding(ni, nf)
    with torch.no_grad(): trunc_normal_(emb.weight, std=0.01)
    return emb

In [86]:
def bn_drop_lin(n_in, n_out, bn=True, p=0., actn=None):
    "Sequence of batchnorm (if `bn`), dropout (with `p`) and linear (`n_in`,`n_out`) layers followed by `actn`."
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

class SimpleNet(nn.Module):
    def __init__(self, in_fts, layers, out_sz, ps=None, use_bn=True, bn_final=False):
        super().__init__()
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        sizes = [in_fts] + layers + [out_sz]
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None]
        layers = []
        
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.layers(x)
        return x

In [87]:
ni = data.train_ds.x.shape[1]
layers = [512, 512, 256, 128]
out_sz = 206

m = SimpleNet(ni, layers, out_sz, ps=0.3)
m

SimpleNet(
  (layers): Sequential(
    (0): Linear(in_features=877, out_features=512, bias=True)
    (1): ReLU(inplace)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU(inplace)
    (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): ReLU(inplace)
    (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.3)
    (12): Linear(in_features=256, out_features=128, bias=True)
    (13): ReLU(inplace)
    (14): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Dropout(p=0.3)
    (16): Linear(in_features=128, out_features=206, bias=True)
  )
)

## Train

In [305]:
def to_np(x): 
    return x.data.cpu().numpy()

def loss_batch(model, x, y, loss_func, opt=None): 
    out = model(x)
    if not loss_func: return to_np(out), to_np(y)
    loss = loss_func(out, y)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.detach().cpu()    
    
def validate(model, dl, loss_fn=None, average=True):
    model.eval()
    with torch.no_grad():
        val_losses,nums = [],[]
#         for xb,yb in tqdm(dl, total=len(dl), unit='batches'):
        for xb,yb in dl:
            val_loss = loss_batch(model, xb, yb, loss_fn)
            val_losses.append(val_loss)
            nums.append(xb.shape[0])
        nums = np.array(nums, dtype=np.float32)
        if average: return (to_np(torch.stack(val_losses)) * nums).sum() / nums.sum()
        else:       return val_losses
        
def fit(model, data, loss_fn, opt, epochs, average=True, scd=None):
    nb = 0
    b2 = opt.param_groups[0]['betas'][1]
    for e in tqdm(range(epochs), total=epochs, unit='epochs'):
        model.train()
        train_losses, nums = [], []
#         for xb,yb in tqdm(data.train_dl, total=len(data.train_dl), unit='batches'):
        for xb,yb in data.train_dl:
            if scd is not None:
                for g in opt.param_groups: g['lr'] = scd[0][nb]
                for g in opt.param_groups: g['betas'] = (scd[1][nb], b2) # only beta1 is scaled
                nb += 1
            loss = loss_batch(model, xb, yb, loss_fn, opt)
            train_losses.append(loss)
            nums.append(xb.shape[0])
        nums = np.array(nums, dtype=np.float32)
        train_loss = (np.stack(train_losses) * nums).sum() / nums.sum()
        valid_loss = validate(model, data.valid_dl, loss_fn, average=True)
        print(f"Epoch {e} -- train_loss: {train_loss}, valid_loss: {valid_loss}")
    print('done!')

def annealing_cos(start, end, pct):
    "Cosine anneal from `start` to `end` as pct goes from 0.0 to 1.0."
    cos_out = np.cos(np.pi * pct) + 1
    return end + (start-end)/2 * cos_out
    
class learner():
    def __init__(self, model, data, loss_fn, opt=optim.Adam):
        self.m, self.data, self.loss_fn = model, data, loss_fn
        self.opt = opt
    
    def OneCycleScheduler(self, epochs, lr, pct_start, moms, div=25):
        final_div=div*1e4
        lr_low=lr/div
        n_batches = len(self.data.train_dl)*epochs
        ph1 = int(n_batches * pct_start)
        ph2 = n_batches-ph1
        
        def steps(start, end, ph1, ph2, final_div):
            up = [annealing_cos(start, end, n/ph1) for n in range(ph1)]
            down = [annealing_cos(end, end/final_div, n/ph2) for n in range(ph2)]
            return up+down
        
        lrs = steps(lr_low, lr, ph1, ph2, final_div)
        moms = steps(moms[0], moms[1], ph1, ph2, final_div)
        return [lrs, moms]
    
    def fit(self, epochs, lr=1e-3, wd=0):
        opt = self.opt(self.m.parameters(), lr=lr, weight_decay=wd)
        fit(self.m, self.data, self.loss_fn, opt, epochs)  

    def fit1cycle(self, epochs, wd=0, lr=1e-2, pct_start=0.3, moms=(0.95,0.85), div=25):
        self.scd = self.OneCycleScheduler(epochs, lr, pct_start, moms, div)
        opt = self.opt(self.m.parameters(), lr=lr, weight_decay=wd)
        fit(self.m, self.data, self.loss_fn, opt, epochs, scd=self.scd)  
        
    def plot_scd(self):
        fig, ax = plt.subplot(1,2)
        ax[0,0] = plt.plot(range(len(self.scd[0])), self.scd[0])
        ax[0,1] = plt.plot(range(len(self.scd[1])), self.scd[1])
        
    def predict(self, dl):
        return validate(self.m, dl, loss_fn=None, average=False)

In [318]:
data = MOA_data(x_fts, y_fts, bs=512)
# data.create(trnval_df_rdy, val_idxs, test=test_df_rdy)
data.create(trt_trnval_df_rdy, val_idxs, test=trt_test_df_rdy)

In [319]:
ni = data.train_ds.x.shape[1]
layers = [512, 512, 256, 128]
out_sz = 206

m = SimpleNet(ni, layers, out_sz, ps=0.3)

In [320]:
learn = None
gc.collect()

loss_func = nn.BCEWithLogitsLoss()
learn = learner(m, data, loss_func)

In [321]:
learn.fit1cycle(10, lr=1e-1)

Epoch 0 -- train_loss: 0.3751540184020996, valid_loss: 0.03696988523006439
Epoch 1 -- train_loss: 0.04447437822818756, valid_loss: 0.03561025112867355
Epoch 2 -- train_loss: 0.023405199870467186, valid_loss: 0.018650054931640625
Epoch 3 -- train_loss: 0.018609484657645226, valid_loss: 0.0177832692861557
Epoch 4 -- train_loss: 0.01785828359425068, valid_loss: 0.01729653589427471
Epoch 5 -- train_loss: 0.017542783170938492, valid_loss: 0.016987888142466545
Epoch 6 -- train_loss: 0.017284123227000237, valid_loss: 0.017164140939712524
Epoch 7 -- train_loss: 0.017046833410859108, valid_loss: 0.016862887889146805
Epoch 8 -- train_loss: 0.016774510964751244, valid_loss: 0.01677710935473442
Epoch 9 -- train_loss: 0.016629096120595932, valid_loss: 0.016783254221081734
done!


In [ ]:
"""
Record: 0.01679310016334057

"""

In [322]:
def unpack(res_list):
    preds = np.vstack([p[0] for p in res_list])
    preds = nn.Sigmoid()(torch.tensor(preds))
    y = np.vstack([p[1] for p in res_list])
    return [preds, y]

def eval_model(learn_obj):
    res = {'train preds': unpack(learn_obj.predict(learn_obj.data.fix_dl)), 
           'valid preds': unpack(learn_obj.predict(learn_obj.data.valid_dl)), 
           'train baseline': [learn_obj.data.train_ds.y, np.zeros(learn_obj.data.train_ds.y.shape)],
           'valid baseline': [learn_obj.data.valid_ds.y, np.zeros(learn_obj.data.valid_ds.y.shape)]}
        
    return res['train preds'], res['valid preds']

In [323]:
train_res, valid_res = eval_model(learn)

In [324]:
train_pred = to_np(train_res[0])
train_y = train_res[1]

valid_pred = to_np(valid_res[0])
valid_y = valid_res[1]

In [325]:
print(f"train log_loss: {metrics.log_loss(train_y, train_pred)}")
print(f"train baseline: {metrics.log_loss(train_y, np.zeros(train_y.shape))}")
print('\n')
print(f"valid log_loss: {metrics.log_loss(valid_y, valid_pred)}")
print(f"valid baseline: {metrics.log_loss(valid_y, np.zeros(valid_y.shape))}")

train log_loss: 2.532457615686388
train baseline: 3.4999552717729197


valid log_loss: 2.634851035657406
valid baseline: 3.412364176996556


In [ ]:
"""
Val record: 2.634851035657406
"""

### Analysis

In [ ]:
def submission_df(preds, ref_df, smplsub):
    cols = smplsub.columns
    res = pd.DataFrame(preds, columns=cols[1:])
    res['sig_id'] = list(ref_df['sig_id'])
    pred_df = res[cols]
    return pred_df

In [ ]:
ref_trn_x = ref_train_x.drop(val_idxs).copy()
ref_val_x = ref_train_x.loc[val_idxs].copy()

train_pred_df = submission_df(train_pred, ref_trn_x, smplsub)
valid_pred_df = submission_df(valid_pred, ref_val_x, smplsub)

In [ ]:
# train
trn_true_y = pd.merge(ref_trn_x[['sig_id', 'cp_type']], ref_train_y, on='sig_id')
trn_pred_y = pd.merge(ref_trn_x[['sig_id', 'cp_type']], train_pred_df, on='sig_id')

ctrl_true = trn_true_y.loc[trn_true_y['cp_type'] == 'ctl_vehicle']
ctrl_pred = trn_pred_y.loc[trn_pred_y['cp_type'] == 'ctl_vehicle']
print('ctrls: ',metrics.log_loss(ctrl_true.iloc[:,2:].to_numpy(), ctrl_pred.iloc[:,2:].to_numpy()))

non_ctrl_true = trn_true_y.loc[trn_true_y['cp_type'] != 'ctl_vehicle']
non_ctrl_pred = trn_pred_y.loc[trn_pred_y['cp_type'] != 'ctl_vehicle']
print('treat:', metrics.log_loss(non_ctrl_true.iloc[:,2:].to_numpy(), non_ctrl_pred.iloc[:,2:].to_numpy()))

In [ ]:
# val
val_true_y = pd.merge(ref_val_x[['sig_id', 'cp_type']], ref_train_y, on='sig_id')
val_pred_y = pd.merge(ref_val_x[['sig_id', 'cp_type']], valid_pred_df, on='sig_id')


val_ctrl_true = val_true_y.loc[val_true_y['cp_type'] == 'ctl_vehicle']
val_ctrl_pred = val_pred_y.loc[val_pred_y['cp_type'] == 'ctl_vehicle']
print('ctrls: ',metrics.log_loss(val_ctrl_true.iloc[:,2:].to_numpy(), val_ctrl_pred.iloc[:,2:].to_numpy()))

val_non_ctrl_true = val_true_y.loc[val_true_y['cp_type'] != 'ctl_vehicle']
val_non_ctrl_pred = val_pred_y.loc[val_pred_y['cp_type'] != 'ctl_vehicle']
print('treat:', metrics.log_loss(val_non_ctrl_true.iloc[:,2:].to_numpy(), val_non_ctrl_pred.iloc[:,2:].to_numpy()))

### Thresholding

In [ ]:
non_ctrl_pred_arr = non_ctrl_pred.iloc[:,2:].to_numpy()
non_ctrl_true_arr = non_ctrl_true.iloc[:,2:].to_numpy()

val_non_ctrl_pred_arr = val_non_ctrl_pred.iloc[:,2:].to_numpy()
val_non_ctrl_true_arr = val_non_ctrl_true.iloc[:,2:].to_numpy()

In [ ]:
def opt_th(targs, preds, start=1e-7, end=1e-5, step=2e-7):
    ths = np.arange(start,end,step)
    res = [metrics.log_loss(targs, (preds > th)*preds) for th in ths]
    idx = np.argmin(res)
    return ths[idx], res[idx]

def ths_binarize(arr, ths):
    arr = arr.copy()
    arr[arr < ths] = 0
    arr[arr > ths] = 1
    return arr

def opt_th_binarize(targs, preds, start=5e-7, end=0.1, step=5e-7):
    ths = np.arange(start,end,step)
    res = [metrics.log_loss(targs, ths_binarize(preds, th)) for th in ths]
    idx = np.argmin(res)
    return ths[idx], res[idx]

In [ ]:
trn_res = opt_th_binarize(non_ctrl_true_arr, non_ctrl_pred_arr)
print(f"Optimal threshold train: {trn_res}")

val_res = opt_th_binarize(val_non_ctrl_true_arr, val_non_ctrl_pred_arr)
print(f"Optimal threshold valid: {val_res}")

In [ ]:
trn_res = opt_th(non_ctrl_true_arr, non_ctrl_pred_arr)
print(f"Optimal threshold train: {trn_res}")

val_res = opt_th(val_non_ctrl_true_arr, val_non_ctrl_pred_arr)
print(f"Optimal threshold valid: {val_res}")